In [ ]:
# Import numpy
import numpy as np
# Import theano and theano.tensor
import theano, theano.tensor as T

In [ ]:
from theano import function, config, shared, sandbox
import theano.tensor as T
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], T.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("First 5 results are %s" % (r[:5]))
if numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

In [ ]:
# Theano on GPU test- warms up GPU nicely....

dim1, dim2 = 15000, 15000
a = np.arange(dim1 * dim2).reshape((dim1,dim2)).astype(np.float32)
b = np.arange(100000,dim1 * dim2 + 100000).reshape((dim2, dim1)).astype(np.float32)

# Load the data into Theano as shared data (note the optional name parameter)
a_t = theano.shared(a, name='a')
b_t = theano.shared(b)

# Use the Theano equivalent of numpy.dot() to get the dot product
ab_t = T.dot(a_t, b_t)
for i in range(1):
    result = ab_t.eval()
# Evaluate to get the result
print(result)

#then release GPU memory
a_t.set_value([[]])
b_t.set_value([[]])

In [ ]:
# keras testing

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import numpy as np
 
data_dim = 20
  
nb_classes = 4
   
model = Sequential()
    
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(64, input_dim=data_dim, init='uniform'))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(64, init='uniform'))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, init='uniform'))
model.add(Activation('softmax'))
     
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
   
# generate dummy training data
x_train = np.random.random((1000, data_dim))
y_train = np.random.random((1000, nb_classes))
   
# generate dummy test data
x_test = np.random.random((100, data_dim))
y_test = np.random.random((100, nb_classes))
      
model.fit(x_train, y_train, nb_epoch=20, batch_size=16)

score = model.evaluate(x_test, y_test, batch_size=16)
print('\nScore:', score)


In [ ]:
# image recognition from pre-trained VGG-19 network

from keras.applications.vgg19 import VGG19
from keras.preprocessing import image as im
from keras.applications.vgg19 import preprocess_input, decode_predictions
from keras.models import Model
from keras.optimizers import SGD
import numpy as np

#model = VGG19(include_top=True, weights='imagenet')

img_path = '../data/dog8.jpg'
img = im.load_img(img_path, target_size=(224, 224))
x = im.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
print('\nPredicted:%s, Probability=%.2f%%' % (decode_predictions(preds)[0], max(preds[0])*100))
#sorted(preds[0], reverse=True)[:5]

In [ ]:
for i in range(1,28):
    try:
        print(i, model.layers[i].get_weights()[0].shape)
    except:
        pass

In [ ]:
model.layers[23].get_config()

In [ ]:
model.output.shape

In [1]:
# ConvNET for Mnist images - from CCA, Deep learning for complete beginners: neural network fine-tuning techniques

from keras.datasets import mnist # subroutines for fetching the MNIST dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Dense, Flatten, Convolution2D, MaxPooling2D, Dropout, merge
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
from keras.regularizers import l2 # L2-regularisation
from keras.layers.normalization import BatchNormalization # batch normalisation
from keras.preprocessing.image import ImageDataGenerator # data augmentation
from keras.callbacks import EarlyStopping # early stopping

batch_size = 128 # in each iteration, we consider 128 training examples at once
num_epochs = 50 # we iterate at most fifty times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth = 32 # use 32 kernels in both convolutional layers
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 128 # there will be 128 neurons in both hidden layers
l2_lambda = 0.0001 # use 0.0001 as a L2-regularisation factor
ens_models = 3 # we will train three separate models on the data

num_train = 60000 # there are 60000 training examples in MNIST
num_test = 10000 # there are 10000 test examples in MNIST

height, width, depth = 28, 28, 1 # MNIST images are 28x28 and greyscale
num_classes = 10 # there are 10 classes (1 per digit)

(X_train, y_train), (X_test, y_test) = mnist.load_data() # fetch MNIST data

X_train = X_train.reshape(X_train.shape[0], depth, height, width)
X_test = X_test.reshape(X_test.shape[0], depth, height, width)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

# Explicitly split the training and validation sets
X_val = X_train[54000:]
Y_val = Y_train[54000:]
X_train = X_train[:54000]
Y_train = Y_train[:54000]

inp = Input(shape=(depth, height, width)) # N.B. Keras expects channel dimension first
inp_norm = BatchNormalization(axis=1)(inp) # Apply BN to the input (N.B. need to rename here)

outs = [] # the list of ensemble outputs
for i in range(ens_models):
    # Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer), applying BN in between
    conv_1 = Convolution2D(conv_depth, kernel_size, kernel_size, border_mode='same', init='he_uniform', W_regularizer=l2(l2_lambda), activation='relu')(inp_norm)
    conv_1 = BatchNormalization(axis=1)(conv_1)
    conv_2 = Convolution2D(conv_depth, kernel_size, kernel_size, border_mode='same', init='he_uniform', W_regularizer=l2(l2_lambda), activation='relu')(conv_1)
    conv_2 = BatchNormalization(axis=1)(conv_2)
    pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
    drop_1 = Dropout(drop_prob_1)(pool_1)
    flat = Flatten()(drop_1)
    hidden = Dense(hidden_size, init='he_uniform', W_regularizer=l2(l2_lambda), activation='relu')(flat) # Hidden ReLU layer
    hidden = BatchNormalization(axis=1)(hidden)
    drop = Dropout(drop_prob_2)(hidden)
    outs.append(Dense(num_classes, init='glorot_uniform', W_regularizer=l2(l2_lambda), activation='softmax')(drop)) # Output softmax layer

out = merge(outs, mode='ave') # average the predictions to obtain the final output

model = Model(input=inp, output=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

datagen = ImageDataGenerator(
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1)  # randomly shift images vertically (fraction of total height)
datagen.fit(X_train)

# fit the model on the batches generated by datagen.flow()---most parameters similar to model.fit
model.fit_generator(datagen.flow(X_train, Y_train,
                        batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=num_epochs,
                        validation_data=(X_val, Y_val),
                        verbose=1,
                        callbacks=[EarlyStopping(monitor='val_loss', patience=5)]) # adding early stopping

model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

Using Theano backend.
Using gpu device 0: GeForce GTX 970 (CNMeM is enabled with initial size: 80.0% of memory, cuDNN not available)
WARNING (theano.gof.compilelock): Overriding existing lock by dead process '10572' (I am process '8968')
DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/Graham/AppData/Local/Theano/compiledir_Windows-10-10.0.14388-SP0-Intel64_Family_6_Model_42_Stepping_7_GenuineIntel-3.5.2-64/tmpz0mz7art/m92a03681cb9b4cade204f4644c19b39f.lib and object C:/Users/Graham/AppData/Local/Theano/compiledir_Windows-10-10.0.14388-SP0-Intel64_Family_6_Model_42_Stepping_7_GenuineIntel-3.5.2-64/tmpz0mz7art/m92a03681cb9b4cade204f4644c19b39f.exp

DEBUG: nvcc STDOUT mod.cu
   Creating library C:/Users/Graham/AppData/Local/Theano/compiledir_Windows-10-10.0.14388-SP0-Intel64_Family_6_Model_42_Stepping_7_GenuineIntel-3.5.2-64/tmpvvev0lwv/m2e0dc00c408313f12b8b63a1376cfc98.lib and object C:/Users/Graham/AppData/Local/Theano/compiledir_Windows-10-10.0.14388-SP0-Intel64_Family_6_Model_4

Epoch 1/50
54000/54000 [==============================] - 71s - loss: 0.4857 - acc: 0.9038 - val_loss: 0.0627 - val_acc: 0.9835
Epoch 2/50
54000/54000 [==============================] - 71s - loss: 0.2783 - acc: 0.9628 - val_loss: 0.0574 - val_acc: 0.9872
Epoch 3/50
54000/54000 [==============================] - 71s - loss: 0.2488 - acc: 0.9699 - val_loss: 0.0492 - val_acc: 0.9873
Epoch 4/50
54000/54000 [==============================] - 71s - loss: 0.2271 - acc: 0.9754 - val_loss: 0.0503 - val_acc: 0.9875
Epoch 5/50
54000/54000 [==============================] - 71s - loss: 0.2231 - acc: 0.9760 - val_loss: 0.0374 - val_acc: 0.9897
Epoch 6/50
54000/54000 [==============================] - 72s - loss: 0.2190 - acc: 0.9785 - val_loss: 0.0340 - val_acc: 0.9912
Epoch 7/50
54000/54000 [==============================] - 71s - loss: 0.2179 - acc: 0.9783 - val_loss: 0.0324 - val_acc: 0.9912
Epoch 8/50
54000/54000 [==============================] - 71s - loss: 0.2154 - acc: 0.9797 - val_loss: 0

[0.027451964621359366, 0.99199999999999999]

In [2]:
model.evaluate(X_test, Y_test, verbose=1)

10000/10000 [==============================] - 5s     


[0.027451964621359366, 0.99199999999999999]